In [2]:
import sys, os
import pandas as pd
import numpy as np
import pickle
import ete3
# per: 
# - https://github.com/etetoolkit/ete/issues/101
# - https://github.com/ContinuumIO/anaconda-issues/issues/1806
os.environ["QT_QPA_PLATFORM"]="offscreen"
os.environ["XDG_RUNTIME_DIR"]="/juno/work/iacobuzc/tmp"
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import logging
from pathlib import Path
# get logger
# make sure to record all logging from modules
logging.basicConfig(stream=sys.stdout, level=logging.INFO)

# clone_color_sequence = sns.cubehelix_palette(n_colors = len(ete_tree.get_leaves()),start=.0, rot=2, light=0.8, dark=0.5, hue=3.5).as_hex()

# del sys.modules['src.tree_io']

from src.tree_io import nx_to_ete_tree, add_info_to_ete_tree
from src.plotting_utils import rgb_string_to_hex
from src.refine_condor_tree import rm_blacklisted_events, find_diploid_clone, merge_clones_into_diploid, adjust_clone_number_and_color
from src.sankoff_fixed_tree import get_sankoff_min_parsimony_tree

import re
from Bio.SeqUtils import seq1

In [3]:
pickle_dir = Path("/home/zhangh5/work/Tapestri_batch2/analysis/condor_pipeline/AJ_results_2023-11-10/pickle_files-nov-10-1")
manual_snv_dir = Path("/home/zhangh5/work/Tapestri_batch2/batch2_data_compiled/manual_annotated_snv_lists")
output_dir = Path("/home/zhangh5/work/Tapestri_batch2/analysis/condor_downstream/HZ_ete_trees_refined_subclonal_snvs")
output_dir.mkdir(exist_ok=True, parents=True)

patient_name = "M04"

f = str(pickle_dir / f'{patient_name}_self.solT_cell')

with open(f, 'rb') as f:
    nx_tree = pickle.load(f)

som_event_dict = {"0": "MISSING", "1": "GAIN", "2": "LOSS", "3": "LOH"}
germ_event_dict = {"0": "MISSING", "2": "LOSS", "3": "LOH"} 

In [4]:
manual_snv_f = manual_snv_dir / f'{patient_name}-patient-all_vars-voi.hz_curated.txt'
manual_snv = pd.read_csv(manual_snv_f, sep='\t', index_col=0, comment = '#')
manual_snv['annotation'].fillna('', inplace=True)
manual_snv['var_formatted'] = manual_snv.index.str.replace(':', '_').str.replace('/', '_')
snv_annotation_map = manual_snv.set_index('var_formatted')['HGVSp'].to_dict()

def __hgvsp_3to1(hgvsp_string):
    """
    1. identify all the 3-letter amino acid codes (3 letters after every capital letter)
    2. replace those 3-letter amino acid codes with 1-letter amino acid codes using Bio.SeqUtils.se1(). Keep other characters intact
    """
    pattern = re.compile(r'[A-Z][a-z][a-z]')
    return re.sub(pattern, lambda x: seq1(x.group()), hgvsp_string)

for k, v in snv_annotation_map.items():
    snv_annotation_map[k] = __hgvsp_3to1(v)

In [5]:
subtrees = {node:ete3.Tree(name=node) for node in nx_tree.nodes()}
[*map(lambda edge:subtrees[edge[0]].add_child(subtrees[edge[1]]), 
nx_tree.edges)]
ete_tree = subtrees['root']

# add two features from nx_tree:
# - cn_profiles
# - cell_attachments
for node in ete_tree.traverse():
    if str(node.name).startswith("root_") or str(node.name).startswith("subtree_"):
        node.delete()
    if nx_tree.nodes()[node.name] != {}:
        for attribute in nx_tree.nodes()[node.name]:
            node.add_feature(attribute, nx_tree.nodes()[node.name][attribute])

In [6]:
for node in ete_tree.traverse():
    if "cell_attachment" in node.features:
        print(node.name)

root
2
5
6
chr18_48584855_ATT_A_4
chr1_27105512_AG_A_4
chr5_68667298_T_G_4
3
chr18_48584804_T_G_4


In [7]:
for node in ete_tree.traverse():
    if "cell_attachment" not in node.features:
        print(node.name)

chr5_38953665_GA_G_1
chr11_92577070_C_T_2
chr8_30933859_T_C_3
chr14_105242966_C_T_2
chr1_16474839_A_AC_3
chr1_226578099_C_T_3
chr13_32907415_A_T_1
chr3_52637486_G_C_3
chr3_30686414_G_A_3
chr17_56448297_T_C_3
chr6_26271232_C_G_3
chr12_4662210_C_T_2
chr12_133252796_G_C_2
chr10_131506192_T_C_2
chr4_96035796_G_GAC_2
chr8_38314915_T_G_3
chr3_30715619_G_C_3
chr8_38314915_T_G_1
chr3_30715619_G_C_1
chr10_131506283_T_C_2
chr13_25052420_A_T_3
chr20_9560633_T_C_2
chr13_25052261_T_C_3
chr3_30715617_T_G_1
chr6_26271232_C_G_2
chr17_73627539_C_T_2
chr20_9560712_G_A_2
chr3_30715617_T_G_3
chr1_158612236_G_A_3


In [8]:
for node in nx_tree.node():
    if "cell_attachment" in nx_tree.nodes()[node]:
        print(node)

AttributeError: 'DiGraph' object has no attribute 'node'

In [9]:
for node in nx_tree.nodes():
    if "cell_attachment" in nx_tree.nodes()[node]:
        print(node)

root
2
3
5
6
chr18_48584804_T_G_4
chr1_27105512_AG_A_4
chr5_68667298_T_G_4
chr18_48584855_ATT_A_4


In [10]:
for node in nx_tree.nodes():
    # if "cell_attachment" in nx_tree.nodes()[node]:
    print(nx_tree.nodes()[node])

{'cell_attachment': ['cell_102-M04-1', 'cell_104-M04-1', 'cell_106-M04-1', 'cell_136-M04-1', 'cell_145-M04-1', 'cell_180-M04-1', 'cell_217-M04-1', 'cell_232-M04-1', 'cell_235-M04-1', 'cell_242-M04-1', 'cell_244-M04-1', 'cell_249-M04-1', 'cell_272-M04-1', 'cell_288-M04-1', 'cell_298-M04-1', 'cell_30-M04-1', 'cell_304-M04-1', 'cell_31-M04-1', 'cell_335-M04-1', 'cell_341-M04-1', 'cell_345-M04-1', 'cell_349-M04-1', 'cell_356-M04-1', 'cell_367-M04-1', 'cell_37-M04-1', 'cell_381-M04-1', 'cell_388-M04-1', 'cell_408-M04-1', 'cell_420-M04-1', 'cell_422-M04-1', 'cell_441-M04-1', 'cell_456-M04-1', 'cell_46-M04-1', 'cell_472-M04-1', 'cell_488-M04-1', 'cell_501-M04-1', 'cell_513-M04-1', 'cell_518-M04-1', 'cell_525-M04-1', 'cell_528-M04-1', 'cell_530-M04-1', 'cell_555-M04-1', 'cell_568-M04-1', 'cell_573-M04-1', 'cell_58-M04-1', 'cell_583-M04-1', 'cell_6-M04-1', 'cell_609-M04-1', 'cell_622-M04-1', 'cell_654-M04-1', 'cell_660-M04-1', 'cell_666-M04-1', 'cell_676-M04-1', 'cell_691-M04-1', 'cell_699-M04-

In [11]:
count = 0 
for node in nx_tree.nodes():
    node_attributes = nx_tree.nodes()[node]
    if "cell_attachment" in node_attributes:
        print(node)
        count += len(node_attributes["cell_attachment"])

root
2
3
5
6
chr18_48584804_T_G_4
chr1_27105512_AG_A_4
chr5_68667298_T_G_4
chr18_48584855_ATT_A_4


In [12]:
count

2769

In [13]:
pickle_dir = Path("/home/zhangh5/work/Tapestri_batch2/analysis/condor_pipeline/AJ_results_2023-11-10/pickle_files-nov-10-1")
manual_snv_dir = Path("/home/zhangh5/work/Tapestri_batch2/batch2_data_compiled/manual_annotated_snv_lists")
output_dir = Path("/home/zhangh5/work/Tapestri_batch2/analysis/condor_downstream/HZ_ete_trees_refined_subclonal_snvs")
output_dir.mkdir(exist_ok=True, parents=True)

patient_name = "M13"

f = str(pickle_dir / f'{patient_name}_self.solT_cell')

with open(f, 'rb') as f:
    nx_tree = pickle.load(f)

som_event_dict = {"0": "MISSING", "1": "GAIN", "2": "LOSS", "3": "LOH"}
germ_event_dict = {"0": "MISSING", "2": "LOSS", "3": "LOH"} 

In [14]:
manual_snv_f = manual_snv_dir / f'{patient_name}-patient-all_vars-voi.hz_curated.txt'
manual_snv = pd.read_csv(manual_snv_f, sep='\t', index_col=0, comment = '#')
manual_snv['annotation'].fillna('', inplace=True)
manual_snv['var_formatted'] = manual_snv.index.str.replace(':', '_').str.replace('/', '_')
snv_annotation_map = manual_snv.set_index('var_formatted')['HGVSp'].to_dict()

def __hgvsp_3to1(hgvsp_string):
    """
    1. identify all the 3-letter amino acid codes (3 letters after every capital letter)
    2. replace those 3-letter amino acid codes with 1-letter amino acid codes using Bio.SeqUtils.se1(). Keep other characters intact
    """
    pattern = re.compile(r'[A-Z][a-z][a-z]')
    return re.sub(pattern, lambda x: seq1(x.group()), hgvsp_string)

for k, v in snv_annotation_map.items():
    snv_annotation_map[k] = __hgvsp_3to1(v)

In [15]:
subtrees = {node:ete3.Tree(name=node) for node in nx_tree.nodes()}
[*map(lambda edge:subtrees[edge[0]].add_child(subtrees[edge[1]]), 
nx_tree.edges)]
ete_tree = subtrees['root']

# add two features from nx_tree:
# - cn_profiles
# - cell_attachments
for node in ete_tree.traverse():
    if str(node.name).startswith("root_") or str(node.name).startswith("subtree_"):
        node.delete()
    if nx_tree.nodes()[node.name] != {}:
        for attribute in nx_tree.nodes()[node.name]:
            node.add_feature(attribute, nx_tree.nodes()[node.name][attribute])

In [16]:
count = 0 
for node in nx_tree.nodes():
    node_attributes = nx_tree.nodes()[node]
    if "cell_attachment" in node_attributes:
        print(node)
        count += len(node_attributes["cell_attachment"])
print(count)

root
0
1
2
6
7
8
chr18_48584804_T_G_4
chr17_7578231_C_CA_4
chr12_49440207_A_C_4
chr19_40741694_T_C_4
chr19_5219803_C_T_4
chr19_5219829_G_T_4
chr15_59322829_G_A_4
3227
